# Projeto 1 - Ciência dos Dados

Nome: Guilherme Martins

Nome: Giovana Cassoni

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [683]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import emoji
from emoji import UNICODE_EMOJI


In [684]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\guilh\OneDrive\Área de Trabalho\Nova pasta (2)\cd\Projeto1_C.Dados-1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [685]:
filename = 'Air Fryer.xlsx'

In [686]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevância
0,air fryer pra canhoto,1
1,@vanessapeitos @daizenbvc o franky conseguiria...,0
2,adoro poder aplicar meus conhecimentos de enge...,1
3,"@leilodecornohu1 ""amigo estamos falando de buc...",0
4,se eu n mandar isso aqui na dm de alguma esqui...,0


In [687]:
probN = train.Relevância.value_counts(True)[0]
probR = train.Relevância.value_counts(True)[1]
train.Treinamento

0                                  air fryer pra canhoto
1      @vanessapeitos @daizenbvc o franky conseguiria...
2      adoro poder aplicar meus conhecimentos de enge...
3      @leilodecornohu1 "amigo estamos falando de buc...
4      se eu n mandar isso aqui na dm de alguma esqui...
                             ...                        
295    @annaborgess air fryer eh foda simplesmente a ...
296    e @chromavka que escolheria brócolis na air fr...
297        estou morando em brasilia ou em uma air fryer
298    queria agradecer ao ser humano que inventou a ...
299    air fryer filha da puta, prega os trem tudo va...
Name: Treinamento, Length: 300, dtype: object

In [688]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevância
0,@poetatulio @tatyene_mendes isso é uma xota ou...,0
1,"comprei uma air fryer p minha mãe, o bagulho caro",0
2,usando air fryer como se não houvesse o amanhã,1
3,minha mãe comprou uma air fryer e já ama mais ...,1
4,repita comigo: churrasco na air fryer não existe!,0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [689]:
import re 


def cleanup(text):
    punctuation = '[@!-.:?;]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [690]:
def cleanup2(text):
    paragrafo = '\n'
    pattern = re.compile(paragrafo)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

In [691]:
def cleanup3(text):
    punctuation = '\/'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

In [692]:
trainR = train.loc[train['Relevância'] == 1,:]
trainN = train.loc[train['Relevância'] == 0,:]

In [693]:
trainR = trainR.Treinamento.apply(str)
trainR = trainR.to_list()

In [694]:
trainN = trainN.Treinamento.apply(str)
trainN = trainN.to_list()

In [695]:
stringR = " "
for i in trainR:
    stringR = stringR + i + ' '
stringR = stringR.lower()
stringR = stringR.replace('air fryer', 'PRODUTO ')
stringR = cleanup3(cleanup2(cleanup(stringR)))
palavrasR = stringR.split()

In [696]:
stringN = " "
for i in trainN:
    stringN = stringN + i + ' '
stringN = stringN.lower()
stringN = stringN.replace('air fryer', 'PRODUTO ')
stringN = cleanup3(cleanup2(cleanup(stringN)))
palavrasN = stringN.split()

In [697]:
serie_tweets_relevantes = pd.Series(palavrasR)
# serie_tweets_relevantes.head(10)

In [698]:
serie_tweets_nao_relevantes = pd.Series(palavrasN)
# serie_tweets_nao_relevantes.head(10)

In [699]:
lista_tweets_relevantes = serie_tweets_relevantes.to_list()
lista_tweets_nao_relevantes = serie_tweets_nao_relevantes.to_list()

In [700]:
tabela_tweets_relevantes = serie_tweets_relevantes.value_counts()
# tabela_tweets_relevantes

In [701]:
tabela_tweets_nao_relevantes = serie_tweets_nao_relevantes.value_counts()
# tabela_tweets_nao_relevantes

In [702]:
tabela_tweets_relevantes_relativa = serie_tweets_relevantes.value_counts(True)
# tabela_tweets_relevantes_relativa

In [703]:
tabela_tweets_nao_relevantes_relativa = serie_tweets_nao_relevantes.value_counts(True)
# tabela_tweets_nao_relevantes_relativa

In [704]:
train_str = train.Treinamento.apply(str)
train = train_str.to_list()
# train

In [705]:
string = " "
for i in train:
    string = string + i + ' '
string = string.lower()
string = string.replace('air fryer', 'PRODUTO ')
string = cleanup3(cleanup2(cleanup(string)))
palavras = string.split()
# palavras

In [706]:
tabela_tweets = pd.Series(palavras)
# tabela_tweets


In [707]:
# tabela_tweets.to_frame()

In [708]:
lista_dos_tweets = []
lista_tweet = []
for tweet in train:
  lista_tweet.append(tweet)
  lista_dos_tweets.append(lista_tweet)
  lista_tweet = []
# lista_dos_tweets


In [709]:
string = " "
for i in train:
    string = string + i + ' '
string = string.lower()
string = string.replace('air fryer', 'PRODUTO ')
string = cleanup3(cleanup2(cleanup(string)))
palavras = string.split()
# palavras


In [710]:
palavras_nao_repetidas = []
for i in palavras:
    if i not in palavras_nao_repetidas:
        palavras_nao_repetidas.append(i)
tabela_nao_repetidas = pd.Series(palavras_nao_repetidas)
# tabela_nao_repetidas

In [711]:
i=0
y = []
while i < 300:
   x = lista_dos_tweets[i]
   x = x[0].lower()
   x = x.replace('air fryer', 'PRODUTO ')
   x = cleanup3(cleanup2(cleanup(x)))
   x=x.split()
   y.append(x)
   i+=1
# y


In [712]:
i = 0
listaR=[]
while i < 300:
    p = y[i]
    probFraseDadoR = 1
    for n in p:
        if n in tabela_tweets_relevantes:
            probFraseDadoR = probFraseDadoR * ((tabela_tweets_relevantes[n]+1)/(len(tabela_tweets_relevantes)+len(tabela_nao_repetidas)))
        else:
            probFraseDadoR = probFraseDadoR * (1/(len(tabela_tweets_relevantes)+len(tabela_nao_repetidas)))

    listaR.append(probFraseDadoR)
    i+=1
# listaR

In [713]:
i = 0
listaN=[]
while i < 300:
    p = y[i]
    probFraseDadoN = 1
    for n in p:
        if n in tabela_tweets_nao_relevantes:
            probFraseDadoN = probFraseDadoN * ((tabela_tweets_nao_relevantes[n]+1)/(len(tabela_tweets_nao_relevantes)+len(tabela_nao_repetidas)))
        else:
            probFraseDadoN = probFraseDadoN * (1/(len(tabela_tweets_nao_relevantes)+len(tabela_nao_repetidas)))

    listaN.append(probFraseDadoN)
    i+=1
# listaN

In [714]:
i = 0
listar=[]
while i < 300:
    probRDadoFrase = listaR[i] * probR
    listar.append(probRDadoFrase)
    i+=1
# listar

In [715]:
i = 0
listan=[]
while i < 300:
    probNDadoFrase = listaN[i] * probN
    listan.append(probNDadoFrase)
    i+=1
# listan

In [716]:
i = 0
relevancia = []
while i < 300:
    if listar[i]>listan[i]:
        relevancia.append('relevante')
    else:
        relevancia.append('irrelevante')
    i+=1
# relevancia


In [717]:
def relevante(lista_de_tweet):
    lista_dos_tweets2 = []
    lista_tweet2 = []
    for tweet in lista_de_tweet:
        lista_tweet2.append(tweet)
        lista_dos_tweets2.append(lista_tweet2)
        lista_tweet2 = []

    i=0
    y1 = []
    while i < 200:
        x = lista_dos_tweets2[i]
        x = x[0].lower()
        x = x.replace('air fryer', 'PRODUTO ')
        x = cleanup3(cleanup2(cleanup(x)))
        x=x.split()
        y1.append(x)
        i+=1
        
    i = 0
    listaR=[]
    while i < 200:        
        p = y1[i]
        probFraseDadoR = 1
        for n in p:
            if n in tabela_tweets_relevantes:
                probFraseDadoR = probFraseDadoR * ((tabela_tweets_relevantes[n]+1)/(len(tabela_tweets_relevantes)+len(tabela_nao_repetidas)))
            else:
                probFraseDadoR = probFraseDadoR * (1/(len(tabela_tweets_relevantes)+len(tabela_nao_repetidas)))

        listaR.append(probFraseDadoR)
        i+=1

    i = 0
    listaN=[]
    while i < 200:       
        p = y1[i]
        probFraseDadoN = 1
        for n in p:
            if n in tabela_tweets_nao_relevantes:
                probFraseDadoN = probFraseDadoN * ((tabela_tweets_nao_relevantes[n]+1)/(len(tabela_tweets_nao_relevantes)+len(tabela_nao_repetidas)))
            else:
                probFraseDadoN = probFraseDadoN * (1/(len(tabela_tweets_nao_relevantes)+len(tabela_nao_repetidas)))

        listaN.append(probFraseDadoN)
        i+=1

    i = 0
    listar=[]
    while i < 200:        
        probRDadoFrase = listaR[i] * probR
        listar.append(probRDadoFrase)
        i+=1

    i = 0
    listan=[]
    while i < 200:
        probNDadoFrase = listaN[i] * probN
        listan.append(probNDadoFrase)
        i+=1

    i = 0
    relevancia = []
    while i < 200:        
        if listar[i]>listan[i]:
            relevancia.append(1)
        else:
            relevancia.append(0)
        i+=1
    return relevancia

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [718]:
test1 = test.Teste.apply(str)
test2 = test1.to_list()





In [725]:
x=relevante(test2)
v=pd.Series(x)
vreal= test.Relevância.to_list()
listaloca=[0]*len(x)
i=0
while i<len(x):
    if x[i] == vreal[i] and x[i] == 0:
        listaloca[i] = 'verdadeiro negativo'
    if x[i] == vreal[i] and x[i] == 1:
        listaloca[i] = 'verdadeiro positivo'

    if x[i] != vreal[i] and x[i] == 0:
        listaloca[i] = 'falso negativo'

    if x[i] != vreal[i] and x[i] == 1:
        listaloca[i] = 'falso positivo'

    i+=1
porcentagens = pd.Series(listaloca)
porcentagens.value_counts(True)*100

verdadeiro positivo    48.5
falso positivo         31.0
verdadeiro negativo    15.0
falso negativo          5.5
dtype: float64

In [724]:
listaloca

['verdadeiro negativo',
 'falso positivo',
 'verdadeiro positivo',
 'verdadeiro positivo',
 'falso positivo',
 'verdadeiro positivo',
 'falso positivo',
 'verdadeiro positivo',
 'verdadeiro positivo',
 'verdadeiro positivo',
 'falso positivo',
 'verdadeiro positivo',
 'verdadeiro positivo',
 'verdadeiro positivo',
 'verdadeiro positivo',
 'verdadeiro positivo',
 'verdadeiro positivo',
 'falso positivo',
 'verdadeiro positivo',
 'verdadeiro positivo',
 'falso positivo',
 'verdadeiro positivo',
 'verdadeiro positivo',
 'verdadeiro negativo',
 'falso positivo',
 'verdadeiro positivo',
 'falso positivo',
 'verdadeiro positivo',
 'falso positivo',
 'falso positivo',
 'verdadeiro positivo',
 'verdadeiro positivo',
 'verdadeiro positivo',
 'verdadeiro positivo',
 'verdadeiro positivo',
 'verdadeiro negativo',
 'falso positivo',
 'falso positivo',
 'verdadeiro positivo',
 'verdadeiro positivo',
 'verdadeiro positivo',
 'verdadeiro positivo',
 'falso negativo',
 'verdadeiro positivo',
 'verdade

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**